# Week 4 — Session 2 (Hands‑On): Wrangling with pandas

This notebook covers:
- NYC Taxi wrangling pipeline
- COVID-19 time series wrangling
- Mini-challenge


## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# pd.set_option("display.max_columns", 50)

## 1. NYC Taxi Wrangling Pipeline
Download the NYC data from the website: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page  
We will use yellow taxi data fror January, 2024

In [ ]:
taxi = pd.read_parquet("~/Desktop/Week4_classroom/yellow_tripdata_2024-01.parquet")

print(taxi.shape)
taxi.head()


In [ ]:
taxi.to_csv("yellow_tripdata_2024-01.tsv", sep= "\t", index=False)

In [ ]:
taxi = pd.read_csv("yellow_tripdata_2024-01.tsv", sep= "\t", parse_dates=["tpep_pickup_datetime","tpep_dropoff_datetime"])

In [ ]:
taxi.columns[6]

In [ ]:
taxi = pd.read_csv("yellow_tripdata_2024-01.tsv", sep= "\t", parse_dates=["tpep_pickup_datetime","tpep_dropoff_datetime"], dtype={"store_and_fwd_flag": str})

In [ ]:
print(taxi.shape)
taxi.head()

In [ ]:
# Clean & feature engineering
taxi_clean = taxi.dropna(subset=["trip_distance"])
taxi_clean = taxi_clean[(taxi_clean.trip_distance.between(2,60)) & (taxi_clean.fare_amount.between(10,250))]
taxi_clean["pickup_hour"] = taxi_clean["tpep_pickup_datetime"].dt.hour
taxi_clean["weekday"] = taxi_clean["tpep_pickup_datetime"].dt.day_name()


In [ ]:
weekday_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
taxi_clean["weekday"] = pd.Categorical(taxi_clean["weekday"], categories=weekday_order, ordered=True)

taxi_clean.head()

## seeking additional information, in this case, the Borough in New York City

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

gdf = gpd.read_file('taxi_zones/taxi_zones.shp')
fig, ax = plt.subplots(1, 1, figsize=(10, 10))


gdf.plot(ax=ax, color='lightgray', edgecolor='black')

plt.axis('off')
plt.show()

gdf.plot()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

gdf = gpd.read_file('taxi_zones/taxi_zones.shp')
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(ax=ax, color='lightgray', edgecolor='black')

# gdf.plot()

# Replace 'label_column' with the name of the column containing your labels
for idx, row in gdf.iterrows():
    # Get the centroid of the geometry for label placement
    # For Point geometries, use row.geometry.x, row.geometry.y
    # For Polygon/LineString geometries, use row.geometry.centroid.x, row.geometry.centroid.y
    x, y = row.geometry.centroid.x, row.geometry.centroid.y
    # Add the label
    ax.annotate(text=row['borough'], xy=(x, y), horizontalalignment='center', fontsize=8, color='darkblue')

plt.axis('off')
plt.show()

In [ ]:
zones = pd.read_csv("taxi_zone_lookup.csv")
zones.head()

In [ ]:
zones['Borough'].unique()

In [ ]:
# Join with the borough
with_boro = taxi_clean.merge(zones[["LocationID","Borough"]], left_on="PULocationID", right_on="LocationID", how="left")
with_boro.head()

In [ ]:
with_boro = taxi_clean.merge(zones[["LocationID","Borough", "Zone"]], left_on="PULocationID", right_on="LocationID", how="left")
with_boro.head()

**filter data based on Borough information**

In [ ]:
nyc_boroughs = ['Queens', 'Manhattan', 'Brooklyn', 'Bronx']


with_boro = with_boro[with_boro['Borough'].isin(nyc_boroughs)]

print(with_boro.shape)
with_boro.head()



## GroupBy function

In [ ]:
# Group & summarize
by_hour = with_boro.groupby("pickup_hour").agg(trips=("trip_distance","count")).reset_index()
by_weekday = with_boro.groupby("weekday").size().rename("trips")
by_hour.head(), by_weekday

In [ ]:
# Visualize
by_hour.plot(x="pickup_hour", y="trips", kind="bar", title="Trips by Hour")
plt.show()


In [ ]:
by_weekday.plot(kind="bar", title="Trips by Weekday")
plt.show()

### Heatmap of mean trip distance by weekday × hour

In [ ]:
pivot_md = with_boro.pivot_table(index="weekday", columns="pickup_hour", values="trip_distance", aggfunc="mean")
plt.imshow(pivot_md.values, aspect="auto")
plt.xticks(range(pivot_md.shape[1]), pivot_md.columns)
plt.yticks(range(pivot_md.shape[0]), pivot_md.index)
plt.colorbar(label="Miles")
plt.title("Mean Trip Distance by Weekday × Hour")
plt.show()

**One step further with population**

In [ ]:
pop = pd.read_csv("nyc_population_2024.csv")
pop.head()

In [ ]:
with_boro_pop = pd.merge(with_boro, pop, on='Borough', how='left')
with_boro_pop.head()

In [ ]:
by_hour = (
    with_boro_pop.groupby(["pickup_hour", "Borough"], as_index=False)
    .agg({"passenger_count": "sum", "pop": "first"})  # pop is constant per borough
)

# Normalize by borough population
by_hour["passenger_per_capita"] = result["passenger_count"] / result["pop"]

print(by_hour.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.lineplot(
    data=result,
    x="pickup_hour",
    y="passenger_per_capita",
    hue="Borough",
    marker="o"
)

plt.title("Per-Capita Taxi Passengers by Hour and Borough (2024)")
plt.xlabel("Pickup Hour")
plt.ylabel("Passengers per Capita")
plt.xticks(range(0,24))
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()


In [ ]:
weekday_result = (
    with_boro_pop.groupby(["weekday", "Borough"], as_index=False)
    .agg({"passenger_count": "sum", "pop": "first"})
)

# Normalize by borough population
weekday_result["passenger_per_capita"] = weekday_result["passenger_count"] / weekday_result["pop"]

print(weekday_result.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.barplot(
    data=weekday_result,
    x="weekday",
    y="passenger_per_capita",
    hue="Borough"
)

plt.title("Per-Capita Taxi Passengers by Weekday and Borough (2024)")
plt.xlabel("Weekday")
plt.ylabel("Passengers per Capita")
plt.legend(title="Borough")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure weekday order
# order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
# weekday_result["weekday"] = pd.Categorical(
#     weekday_result["weekday"], categories=order, ordered=True
# )

# Pivot to have boroughs as rows, weekdays as columns
heatmap_data = weekday_result.pivot(index="Borough", columns="weekday", values="passenger_per_capita")


In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(
    heatmap_data,
    annot=True, fmt=".6f", cmap="RdBu", linewidths=0.5, cbar_kws={'label': 'Passengers per Capita'}
)

plt.title("Per-Capita Taxi Passengers by Weekday and Borough (2024)")
plt.xlabel("Weekday")
plt.ylabel("Borough")
plt.show()


In [ ]:
# Pivot to create heatmap table
heatmap_hourly = by_hour.pivot(
    index="Borough", 
    columns="pickup_hour", 
    values="passenger_per_capita"
)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14,6))
sns.heatmap(
    heatmap_hourly,
    annot=False, cmap="coolwarm", linewidths=0.5, cbar_kws={'label': 'Passengers per Capita'}
)

plt.title("Per-Capita Taxi Passengers by Hour and Borough (2024)")
plt.xlabel("Pickup Hour")
plt.ylabel("Borough")
plt.xticks(rotation=0)
plt.show()


Color palettes: Seaborn: https://seaborn.pydata.org/tutorial/color_palettes.html  
matlibplot: https://matplotlib.org/stable/users/explain/colors/colormaps.html


## 2. COVID‑19 Time Series Wrangling

In [ ]:
covid = pd.read_csv("covid_daily.csv", parse_dates=["date"])
ny = covid[covid.state=="NY"].set_index("date").sort_index()
ny["new_cases"] = ny["cases"].diff()
weekly = ny["new_cases"].resample("W-SUN").sum()
rolling = ny["new_cases"].rolling(7, min_periods=1).mean()
weekly.plot(title="NY New Cases Weekly Sum")
plt.show()
rolling.plot(title="NY New Cases 7-day Rolling Mean")
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1) Load and filter NY data ---
covid = pd.read_csv("covid_daily.csv", parse_dates=["date"])
ny = (covid[covid["state"] == "NY"]
      .set_index("date")
      .sort_index())

# Daily new cases from cumulative
ny["new_cases"] = ny["cases"].diff()

# --- 2) Compute weekly sum ---
weekly = ny["new_cases"].resample("W-SUN").sum().reset_index()
weekly.columns = ["date", "weekly_cases"]

# --- 3) Compute 7-day rolling mean ---
rolling = ny["new_cases"].rolling(7, min_periods=1).mean().reset_index()
rolling.columns = ["date", "rolling_mean"]

# --- 4) Plot weekly sum ---
plt.figure(figsize=(12,6))
sns.lineplot(data=weekly, x="date", y="weekly_cases", marker="o", color="steelblue")
plt.title("NY New COVID-19 Cases: Weekly Sum")
plt.xlabel("Date")
plt.ylabel("Weekly New Cases")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- 5) Plot rolling mean ---
plt.figure(figsize=(12,6))
sns.lineplot(data=rolling, x="date", y="rolling_mean", color="darkorange")
plt.title("NY New COVID-19 Cases: 7-day Rolling Mean")
plt.xlabel("Date")
plt.ylabel("Daily New Cases (7-day avg)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 3. Mini‑Challenge

In [ ]:
# Mean fare by borough × hour
mean_fare = ???
pivot_fare = ???
plt.imshow(pivot_fare.values, aspect="auto")
plt.xticks(range(pivot_fare.shape[1]), pivot_fare.columns)
plt.yticks(range(pivot_fare.shape[0]), pivot_fare.index)
plt.colorbar(label="$")
plt.title("Mean Fare by Borough × Hour")
plt.show()

In [ ]:
# Bonus: trips by passenger count
by_passengers = ??
by_passengers.plot(kind="bar", title="Trips by Passenger Count")
plt.show()